In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

C:\ProgramData\Anaconda3\envs\coursera_ds\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\envs\coursera_ds\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
data = pd.read_csv('../../data/telecom_churn.csv')

In [4]:
data.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [6]:
# Проводим предобработку датафрейма
# Чтобы обучать модели в slkearn,нам нужно их перевести их в числа, поэтому
# убираем категориальный признак State и убираем неважный бинарный признак
#Voice mail plan, из прошлых занятий мы помним что он не важный.
data.drop(['State','Voice mail plan'], axis=1, inplace=True)
# Напомним что по умолчанию удаляются строки axis=0,чтобы удалить столбец 
# нужно указать axis=1, Так же что бы изменился исходный датафрейм
# Указываем параметр inplace=True

In [7]:
# У нас есть важный признак International plan, он бинарный строковый
# Нам необходимо перевести его в числовой(вида 0 или 1). Сделаем это с помощью
# map
data['International plan'] = data['International plan'].map({'Yes':1,'No':0})

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 18 columns):
Account length            3333 non-null int64
Area code                 3333 non-null int64
International plan        3333 non-null int64
Number vmail messages     3333 non-null int64
Total day minutes         3333 non-null float64
Total day calls           3333 non-null int64
Total day charge          3333 non-null float64
Total eve minutes         3333 non-null float64
Total eve calls           3333 non-null int64
Total eve charge          3333 non-null float64
Total night minutes       3333 non-null float64
Total night calls         3333 non-null int64
Total night charge        3333 non-null float64
Total intl minutes        3333 non-null float64
Total intl calls          3333 non-null int64
Total intl charge         3333 non-null float64
Customer service calls    3333 non-null int64
Churn                     3333 non-null bool
dtypes: bool(1), float64(8), int64(9)
memory usage

In [10]:
# Выделяем целевой признак, то есть тот признак значение 
#которого мы пытаемся предсказывать
# по традиции его обозначают как у
# Также приведем его к типу инт
y = data['Churn'].astype('int')


In [11]:
#  Определяем матрицу X
# Удаляем целевой признак Churn
X = data.drop('Churn',axis=1)

In [12]:
# Проверяем размерность объектов
X.shape,y.shape

((3333, 17), (3333,))

# Деревья
#### Разбиение датафрейма на основную и проверочную части
в пропорции 70% на 30%

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      test_size=0.3,
                                                      random_state=17 )
# train_test_split разбивает передаваемые фреймы на 2 части, можно передать сразу
# 2 датафрейма(как в примере) и тогда он вернет 4 датафрейма
# test_size отвечает за размер проверочной части
# random_state отвечает за то как будет происходить разбиение(способ)


In [15]:
# Проверяем как произошло разбиение
# Видим что в проверочной части 30% значений исходного фрейма
X_train.shape, X_valid.shape

((2333, 17), (1000, 17))

In [17]:
# Создадим наше первое дерево
# Для это создадим экземпляр класса DecisionTreeClassifier
# Для воспроизодимости следует указывать явно значение параметра random_state
first_tree = DecisionTreeClassifier(random_state=17)

In [21]:
# Импортируем функцию котороя определяет качество кросс-валидации
from sklearn.model_selection import cross_val_score
import numpy as np

In [20]:
cross_val_score (first_tree,X_train,y_train,cv=5)

# Передаем в функцию первым аргументом созданное дерево
# и 2 выборки на которых мы тренируемся
# отвечает за значение кросс-валидации(т.е сколько раз будет происходить
# кросс-валидация. Т.е 5 раз дерево first_tree обучилось на  80% X_train)
# Массив из 5 чисел, это доля верных ответов на 20% данных 
# Нужно смотреть чтобы эти числа были более или менее похожи
# Если есть сильно отличающиеся данные значит что то пошло не так
# 

array([0.9143469 , 0.91220557, 0.92077088, 0.90772532, 0.91416309])

In [22]:
# Вычиляем среднее качество на кросс-валидации
np.mean(cross_val_score (first_tree,X_train,y_train,cv=5)
)

0.9138423504976518

### Метод ближайших соседей(KNN)

In [23]:
# импортируем инструменты
from sklearn.neighbors import KNeighborsClassifier

In [24]:
# Создаем экземпляр класса
first_knn = KNeighborsClassifier()
# Важные параметры
# n_neighbors-это число соседей
# weights -будут ли веса одинаковыми или зависеть от расстояния

In [25]:
# Вычисляем среднее качество кросс-валидации 
# при использовании метода ближайших соседей
np.mean(cross_val_score (first_knn,X_train,y_train,cv=5)
)

0.8671274043984523

Мы видим что метод ближайших соседей выдал низкий по сравнению с деревом результат. Однако не надо торопится. Сначала нужно настроить кросс-валидацию(подберем гиперпараметры) на каждый из способов и только после этого говорить о разнице

**Сначала подберем гиперпараметры для дерево(max_depths и прочее)**

In [27]:
from sklearn.model_selection import GridSearchCV

**GridSearchCV**по сути это решетка из гиперпараметров, она может быть многомерной. Она образует декартово произведение(т.е.  все возможные комбинации гиперпараметров), что найти наилучшее значение кросс-валидации.

По сути этот метод перебирает каждую возможную комбинацию гиперпараметров. Очевидно что на больших объемах данных это будет очень дорого.

In [32]:
# Создаем словарь параметров
tree_params = {'max_depth':np.arange(1,11)}
# мы создали список от 1 до 10 
# Создаем экземпляр класса
tree_grid = GridSearchCV(first_tree, tree_params,cv=5,n_jobs=-1)
# cv  это сколько раз делать кросс-валидацию
# n_jobs -отвечает за распаралеливание, -1 значит что будут использованы все ядра
# Экземпляр класс не начнет свою работу пока мы не передадим туда какие то данные

In [33]:
%%time
tree_grid.fit(X_train, y_train)

Wall time: 3.11 s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=17,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
             pre_dispatch='2*n_jobs', refit=True, re

In [34]:
# Теперь наш обьект tree_grid обучился и нашел лучшие сочетания параметров
tree_grid.best_params_

{'max_depth': 6}

In [35]:
tree_grid.best_score_

0.9417059579939991

Теперь подберем лучшие параметры для метода ближайших соседей


In [37]:
knn_params = {'n_neighbors':[1,2,3,4,5] + list(range(50,100,10))}
knn_grid = GridSearchCV(first_knn,knn_params,cv=5)

In [ ]:
knn_grid = GridSearchCV(first_knn,knn_params,cv=5,n_jobs=-1)

In [39]:
%%time
knn_grid.fit(X_train,y_train);

Wall time: 2.71 s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 50, 60, 70, 80, 90]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [40]:
knn_grid.best_score_,knn_grid.best_params_

(0.8671238748392628, {'n_neighbors': 5})

In [41]:
# Попробуем с другим количеством ближайщих сосоедей
knn_params = {'n_neighbors':list(range(5,30,5))}
knn_grid = GridSearchCV(first_knn,knn_params,cv=5)

In [42]:
knn_grid = GridSearchCV(first_knn,knn_params,cv=5,n_jobs=-1)

In [43]:
%%time
knn_grid.fit(X_train,y_train);

Wall time: 12.7 s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid={'n_neighbors': [5, 10, 15, 20, 25]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [44]:
knn_grid.best_score_,knn_grid.best_params_

(0.8701243034719246, {'n_neighbors': 10})

Мы увидели что на этих данных **дерево** будет лучшим вариантом

In [46]:
# Теперь используем наше обученное дерево на отложенной выборке
# в обьекте грид в параметре best_escimate и хранится наше обученое дерево с лучшими параметрами
# Прогноз осуществляется с помощью метода predict
tree_valid_pred = tree_grid.predict(X_valid)

In [47]:
#  Лучше явно показать какую метрику для оценки мы используем
from sklearn.metrics import accuracy_score
# accuracy_score- показывает долю верных ответов
accuracy_score(y_valid,tree_valid_pred)
# первый аргумент это реальные данные, второй прогнозные
# как видим совпадение почти 95%

0.946

In [49]:
# Базовая проверка на адекватность
# Это доля клиентов которые остаются
# То есть модель которая всегда прогнозирует  что клиент хороший, будет верно
# указывать это с вероятностью 85,5
1- np.mean(y)
# У нашего дерева решений точность равна 94.6%, что очевидно лучше


0.8550855085508551

#### Рисуем графики


In [50]:
from sklearn.tree import export_graphviz

In [51]:
# используем наше обученное дерево
export_graphviz(tree_grid.best_estimator_,out_file='telecom.dot',
               feature_names=X.columns,filled=True )

In [52]:
!dot

"dot" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.
